In [ ]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
#from my_func import *
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble  import  RandomForestRegressor
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from sklearn.decomposition import  *
from sklearn.pipeline import *
from sklearn.grid_search import *
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics.pairwise import *
#from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random
random.seed(2016)


In [ ]:
df_all = pd.read_csv('df_all_feat.csv', encoding="ISO-8859-1")

In [ ]:
df_train = pd.read_csv('input/train.csv', encoding="ISO-8859-1")
num_train = df_train.shape[0]

In [ ]:
print (num_train)

In [ ]:
df_test = pd.read_csv('input/test.csv', encoding="ISO-8859-1")
df_pro_desc = pd.read_csv('input/product_descriptions.csv')
df_attr = pd.read_csv('input/attributes.csv')
#df_brand = pd.read_csv('df_all_brand.csv')
df_brand = df_attr[df_attr.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})

df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')
print("--- Files Loaded: %s minutes ---" % round(((time.time() - start_time)/60),2))



In [ ]:
df_pro_desc = pd.read_csv('input/product_descriptions.csv')

In [ ]:
def str_stem(s): 
    s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s)
    return s

In [ ]:
df_pro_tmp = str_stem(df_pro_desc['product_description'][0])

In [ ]:
strNum = {'zero':0,'one':1,'two':2,'three':3,'four':4,'five':5,'six':6,'seven':7,'eight':8,'nine':9}
def str_stem(s): 
    if isinstance(s, str):
        #s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s)
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
        s = s.lower()
        
        s = s.replace(","," ") #could be number / segment later
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("'"," ")
        s = s.replace(":"," ")
        s = s.replace("."," . ")
        s = s.replace('"', "")
        s = s.replace("("," ")
        s = s.replace(")"," ")
        s = s.replace(";"," ")
        s = s.replace("&"," ")
        s = s.replace("!"," ")
        s = s.replace("#"," ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = s.replace(" x "," xbi ")
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = s.replace("°"," degrees ")
        s = re.sub(r"([0-9]+)( *)(degrees|degree)\.?", r"\1deg. ", s)
        s = s.replace(" v "," volts ")
        s = re.sub(r"([0-9]+)( *)(volts|volt)\.?", r"\1volt. ", s)
        s = re.sub(r"([0-9]+)( *)(watts|watt)\.?", r"\1watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1amp. ", s)

        s = s.replace("children", "kid ")
        s = s.replace("child", "kid ")
        s = s.replace("kids", "kid ")
        s = s.replace("refrigerators", "fridge ")
        s = s.replace("refrigerator", "fridge ")
        s = s.replace("freezers", "fridge ")
        s = s.replace("freezer", "fridge ")
        s = s.replace("fragrances", "perfume ")
        s = s.replace("fragrance", "perfume ")
        s = s.replace("perfumes", "perfume ")
        s = s.replace("coffeemaker", "coffee maker")
        #s = (" ").join([z for z in s.split(" ") if z not in stop_w])
        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        
        #s = s.lower()
        s = s.replace("toliet","toilet")
        s = s.replace("airconditioner","air conditioner")
        s = s.replace("vinal","vinyl")
        s = s.replace("vynal","vinyl")
        s = s.replace("skill","skil")
        s = s.replace("snowbl","snow bl")
        s = s.replace("plexigla","plexi gla")
        s = s.replace("rustoleum","rust-oleum")
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool ga")
        s = s.replace("whirlpoolstainless","whirlpool stainless")

        s = s.replace(" . "," ")
        s = s.replace("  "," ")
        s = s.replace("  "," ")
        return s
    else:
        return "null"

def seg_words(str1, str2):
    str2 = [z for z in set(str2.split()) if len(z)>2]
    words = str1.lower().split(" ")
    s = []
    for word in words:
        if len(word)>3:
            s1 = []
            s1 += segmentit(word,str2,True)
            if len(s)>1:
                s += [z for z in s1 if z not in ['er','ing','s','less'] and len(z)>1]
            else:
                s.append(word)
        else:
            s.append(word)
    return (" ".join(s))

def segmentit(s, txt_arr, t):
    st = s
    r = []
    for j in range(len(s)):
        for word in txt_arr:
            if word == s[:-j]:
                r.append(s[:-j])
                #print(s[:-j],s[len(s)-j:])
                s=s[len(s)-j:]
                r += segmentit(s, txt_arr, False)
    if t:
        i = len(("").join(r))
        if not i==len(st):
            r.append(st[i:])
    return r

def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

def str_brand(str1, str2):
    if str1 == "null":
        for word in str2:
            for b in brand:
                if word in b and b in str2:
                    return b
    return str1


In [ ]:
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))

In [ ]:
df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')

In [ ]:
df_all['product_description'] = df_all['product_description'].map(lambda x:repl(x))

In [ ]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']

In [ ]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))
#df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
df_all['brand'] = df_brand['brand']
print("--- Stemming: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']
print("--- Prod Info: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)
print("--- Len of: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['search_term'] = df_all['product_info'].map(lambda x:seg_words(x.split('\t')[0],x.split('\t')[1]))
#print("--- Search Term Segment: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))
df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
print("--- Query In: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['query_last_word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[1]))
df_all['query_last_word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[2]))
print("--- Query Last Word In: %s minutes ---" % round(((time.time() - start_time)/60),2))
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel())
d={}
i = 1000
for s in df_brand:
    d[s]=i
    i+=3
df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))
#df_all.to_csv('df_all.csv')

In [ ]:
df_query= pd.unique(df_all.search_term.ravel())
q={}
i = 1
for s in df_query:
    q[s]=i
    i+=3
df_all['query_feature'] = df_all['search_term'].map(lambda x:q[x])    


In [ ]:
for s in df_brand:
    d[s]=i
    i+=3

In [ ]:
tfvect = TfidfVectorizer(use_idf=True, stop_words = 'english')
wholeword = df_all['search_term']+" "+df_all['product_title'] +" "+df_all['product_description']+ " "+df_all['product_attributes']

In [ ]:
vocab = tfvect.fit_transform(wholeword)

In [ ]:
st = tfvect.transform(df_all['search_term'])
pt = tfvect.transform(df_all['product_title'])
pde = tfvect.transform(df_all['product_description'])
pa = tfvect.transform(df_all['product_attributes'])

In [ ]:
print (st.shape[0])

In [ ]:
df_all['cossim_query_title'] = list(map(lambda x,y: cosine_similarity(x, y), st, pt))
cosime_q_t = df_all['cossim_query_title'].astype('float64', coerce=True)
df_all['cossim_query_title']=cosime_q_t

In [ ]:
df_all['cossim_query_description'] = list(map(lambda x,y: cosine_similarity(x, y), st, pde))
consime_q_de = df_all['cossim_query_description']
df_all['cossim_query_description'] = consime_q_de.astype('float64', coerce = True)

In [ ]:
df_all['cossim_query_attribute'] = list(map(lambda x,y: cosine_similarity(x, y), st, pa))
consime_q_at = df_all['cossim_query_attribute']
df_all['cossim_query_attribute'] = consime_q_at.astype('float64', coerce = True)

In [ ]:
df_all=df_all.drop(['product_info','attr'],axis=1)

In [ ]:
df_all

In [ ]:
def try_divide(x, y, val=0.0):
    if y != 0.0:
        val = float(x) / y
    return val
def JaccardCoef(A, B):
    A, B = set(A), set(B)
    intersect = len(A.intersection(B))
    union = len(A.union(B))
    coef = try_divide(intersect, union)
    return coef

def DiceDist(A, B):
    A, B = set(A), set(B)
    intersect = len(A.intersection(B))
    union = len(A) + len(B)
    d = try_divide(2*intersect, union)
    return d

def compute_dist(A, B, dist="jaccard_coef"):
    if dist == "jaccard_coef":
        d = JaccardCoef(A, B)
    elif dist == "dice_dist":
        d = DiceDist(A, B)
    return d

In [ ]:
df_all['jaccardcoef_query_title'] = list(df_all.apply(lambda x: compute_dist(x['search_term'], x['product_title'], "jaccard_coef"), axis=1))

In [ ]:
df_all['jaccardcoef_query_attribute'] = list(df_all.apply(lambda x: compute_dist(x['search_term'], x['product_attributes'], "jaccard_coef"), axis=1))

In [ ]:
pd.DataFrame({ "dist": df_all['jaccardcoef_query_attribute']}).to_csv('jaccardcoef_query_attr.csv',index=False)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def cs_retr(x1, x2):
    return cosine_similarity(x1, x2)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'a' :[st], 'b' : [pt], 'c' : [pde], 'd' : [pa]})

In [ ]:
df['st-pt'] = df['a'].map(lambda x: cs_retr(, x['b']), df)

In [ ]:
print (df_all['cos_sim_st_pt'][:10])

In [ ]:
df_attr = pd.read_csv("input/attributes.csv", encoding="ISO-8859-1")
df_attr.dropna(how="all", inplace=True)
df_attr["product_uid"] = df_attr["product_uid"].astype(int)
df_attr["value"] = df_attr["value"].astype(str)

def concate_attrs(attrs):
    names = attrs["name"]
    values = attrs["value"]
    pairs  = []
    for n, v in zip(names, values):
        pairs.append(' '.join((n, v)))
    return ' '.join(pairs)

product_attrs = df_attr.groupby("product_uid").apply(concate_attrs)

product_attrs = product_attrs.reset_index(name="product_attributes")




In [ ]:
def cooccurrence_terms(lst1, lst2, join_str):
    terms = [""] * len(lst1) * len(lst2)
    cnt =  0
    for item1 in lst1:
        for item2 in lst2:
            terms[cnt] = item1 + join_str + item2
            cnt += 1
    res = " ".join(terms)
    return res

In [ ]:
join_str = "X"
df_all["query_title"] = list(df_all.apply(lambda x: cooccurrence_terms(x["query_unigram"], x["title_unigram"], join_str), axis=1))

In [ ]:
token_pattern = r"(?u)\b\w\w+\b"
def preprocess_data(line,
                    token_pattern=token_pattern,
                    encode_digit=False):
    token_pattern = re.compile(token_pattern, flags = re.UNICODE | re.LOCALE)
    ## tokenize
    tokens = [x.lower() for x in token_pattern.findall(line)]
    return tokens

In [ ]:
df_all["query_unigram"] = list(df_all.apply(lambda x: preprocess_data(x["search_term"]), axis=1))
#df_all["title_unigram"] = list(df_all.apply(lambda x: preprocess_data(x["product_title"]), axis=1))

In [ ]:
df_all["description_unigram"] = list(df_all.apply(lambda x: preprocess_data(x["product_description"]), axis=1))
df_all["attribute_unigram"] = list(df_all.apply(lambda x: preprocess_data(x["product_attributes"]), axis=1))

In [ ]:
df_all["query_description"] = list(df_all.apply(lambda x: cooccurrence_terms(x["query_unigram"], x["description_unigram"], join_str), axis=1))
df_all["query_attribute"] = list(df_all.apply(lambda x: cooccurrence_terms(x["query_unigram"], x["attribute_unigram"], join_str), axis=1))

In [ ]:
df_all=df_all.drop(['description_unigram','attribute_unigram'],axis=1)

In [ ]:
q_de = df_all["query_description"] 
q_attr = df_all["query_attribute"]

In [ ]:
df_all["query_description"] = q_de
df_all["query_attribute"] = q_attr

In [ ]:
print (df_all.info())

In [ ]:
print (df_all[:5])

In [ ]:
df_all=df_all.drop(['query_unigram','title_unigram'],axis=1)

In [ ]:
df_all = pd.merge(df_all, product_attrs, how="left", on="product_uid")
df_all['product_attributes'] = df_all['product_attributes'].fillna('na')

In [ ]:
df_all['product_attributes'] = df_all['product_attributes'].map(lambda x:repl(x))

In [ ]:
df_all['product_info'] = df_all['search_term']+"\t"+df_all['product_title'] +"\t"+df_all['product_description']+"\t"+df_all['product_attributes']
df_all['len_of_attributes'] = df_all['product_attributes'].map(lambda x:len(x.split())).astype(np.int64)
print ("len")
df_all['query_in_attributes'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[3],0))
print ("query in attr")
df_all['word_in_attributes'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[3]))
print ("word in ")
df_all['ratio_attributes'] = df_all['word_in_attributes']/df_all['len_of_query']
print ("ratio in ")
df_all['query_last_word_in_attributes'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0].split(" ")[-1],x.split('\t')[3]))

In [ ]:
print (df_all['product_description'][0])

In [ ]:
df_all.to_csv('df_all_feat.csv',index=False)

In [ ]:
product_counts = pd.DataFrame(pd.Series(df_all.groupby(["product_uid"]).size(), name="product_count"))
df_all = pd.merge(df_all, product_counts, left_on="product_uid", right_index=True, how="left")

In [ ]:
#df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train =df_train[:]
X_test = df_test[:]
print("--- Features Set: %s minutes ---" % round(((time.time() - start_time)/60),2))

In [ ]:
print (df_train[-5:])

In [ ]:
nltk.download() 

In [ ]:
sentences = []  # Initialize an empty list of sentences

for info in df_all['search_term']:
    sentences += info_to_sentences(info, tokenizer)
    
for info in df_all['product_title']:
    sentences += info_to_sentences(info, tokenizer)
    
for info in df_all['product_description']:
    sentences += info_to_sentences(info, tokenizer)


In [ ]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
#num_features = 3000    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
#downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, min_count = min_word_count, window = context)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()

In [ ]:
model.syn0.shape

In [ ]:
print (df_all.shape[0])

In [ ]:
all_tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1")
all_word = df_all['search_term'] + " " + df_all['product_title'] + " " + df_all['product_description']
all_tfidf.fit_transform(all_word)
        
class cust_tfidf_vals(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):        
        return all_tfidf.transform(data_dict[self.key])

In [ ]:

clean_train_query = []
for info in X_train["search_term"]:
    clean_train_query.append( info_to_wordlist( info, remove_stopwords=False ))

trainDataVecs = getAvgFeatureVecs( clean_train_query, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_query = []
for info in X_test["search_term"]:
    clean_test_query.append( info_to_wordlist( info, remove_stopwords=False ))

testDataVecs = getAvgFeatureVecs( clean_test_query, model, num_features )

In [ ]:
forest = RandomForestRegressor(n_estimators = 500, n_jobs = -1, random_state = 2016, verbose = 1)
forest = forest.fit( trainDataVecs, y_train)

# Test & extract results 
result = forest.predict( testDataVecs )

pd.DataFrame({"id": id_test, "relevance": result}).to_csv('submission.csv',index=False)

In [ ]:
from my_func import *
print ("finish import")

In [ ]:
from imp import reload
import my_func
reload(my_func)

In [ ]:
start_time = time.time()
print("--- Training Start ---")
rfr = RandomForestRegressor(n_estimators = 500, n_jobs = -1, random_state = 2016, verbose = 1)
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words='english')
tsvd = TruncatedSVD(n_components=10, random_state = 2016)
clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                        #('feat1', cust_number_col(key='query_in_title')),
                        #('feat2', cust_number_col(key='query_in_description')),
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        #('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)])),
                        ('txt5', pipeline.Pipeline([('s5', cust_txt_col(key='query_title')), ('tfidf5', tfidf), ('tsvd5', tsvd)])),
                        ('txt6', pipeline.Pipeline([('s6', cust_txt_col(key='query_description')), ('tfidf6', tfidf), ('tsvd6', tsvd)])),
                        #('txt7', pipeline.Pipeline([('s7', cust_txt_col(key='query_attribute')), ('tfidf7', tfidf), ('tsvd7', tsvd)])),
                        #('txt5', pipeline.Pipeline([('s5', cust_txt_col(key='product_attributes')), ('tfidf5', tfidf), ('tsvd5', tsvd)])),
                        ('txt11', pipeline.Pipeline([('s11', cust_tfidf_vals(key='search_term')), ('tsvd11', tsvd)])),
                        ('txt12', pipeline.Pipeline([('s12', cust_tfidf_vals(key='product_title')), ('tsvd12', tsvd)])),
                        ('txt13', pipeline.Pipeline([('s13', cust_tfidf_vals(key='product_description')), ('tsvd13', tsvd)]))              
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        #'feat1':0.3,
                        #'feat2':0.3,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        #'txt3': 0.0,
                        'txt4': 0.5,
                        'txt5': 1.0,
                        'txt6': 1.0,
                        #'txt7': 1.0,
                        'txt11': 0.1,
                        'txt12': 0.1,
                        'txt13': 0.05,
                        },
                n_jobs = -1
                )), 
        ('rfr', rfr)
    ])
param_grid = {'rfr__max_features': [10], 'rfr__max_depth': [20]}
RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 2, verbose = 20, scoring=RMSE)
model.fit(X_train, y_train)

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)
print(model.best_score_ + 0.454686256525)

y_pred = model.predict(X_test)
#y_pred[y_pred>3]=3
#y_pred[y_pred<1]=1
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)
print("--- Training & Testing: %s minutes ---" % round(((time.time() - start_time)/60),2))



In [ ]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def info_to_wordlist( info, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    #  
    # 2. Remove non-letters
    info = re.sub("[^a-zA-Z0-9]"," ", info)
    #
    # 3. Convert words to lower case and split them
    words = info.split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [ ]:
print (model.grid_scores_)

In [ ]:
from sklearn.feature_extraction import text
import nltk

df_outliers = pd.read_csv('df_all.csv', encoding="ISO-8859-1", index_col=0)
#stop_ = list(text.ENGLISH_STOP_WORDS)
stop_ = []
d={}
for i in range(len(df_outliers)):
    s = str(df_outliers['search_term'][i]).lower()
    #s = s.replace("\n"," ")
    #s = re.sub("[^a-z]"," ", s)
    #s = s.replace("  "," ")
    a = set(s.split(" "))
    for b_ in a:
        if b_ not in stop_ and len(b_)>0:
            if b_ not in d:
                d[b_] = [1,str_common_word(b_, df_outliers['product_title'][i]),str_common_word(b_, df_outliers['brand'][i]),str_common_word(b_, df_outliers['product_description'][i])]
            else:
                d[b_][0] += 1
                d[b_][1] += str_common_word(b_, df_outliers['product_title'][i])
                d[b_][2] += str_common_word(b_, df_outliers['brand'][i])
                d[b_][3] += str_common_word(b_, df_outliers['product_description'][i])
ds2 = pd.DataFrame.from_dict(d,orient='index')
ds2.columns = ['count','in title','in brand','in prod']
ds2 = ds2.sort_values(by=['count'], ascending=[False])

f = open("word_review.csv", "w")
f.write("word|count|in title|in brand|in description\n")
for i in range(len(ds2)):
    f.write(ds2.index[i] + "|" + str(ds2["count"][i]) + "|" + str(ds2["in title"][i]) + "|" + str(ds2["in brand"][i]) + "|" + str(ds2["in prod"][i]) + "\n")
f.close()
print("--- Word List Created: %s minutes ---" % round(((time.time() - start_time)/60),2))

In [ ]:
import gensim